In [ ]:
from nbdev import *
%nbdev_default_export experiments

Cells will be exported to mlchaos.experiments,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# Experiments

> Experiments to be run as CLI scripts (too long to run in a Jupyter session)

In [ ]:
%nbdev_export
import papermill as pm
import numpy as np
from fastscript import *
from fastcore.utils import Path

### Analyse the impact of the number of points on the performance of the motion classifier

The results will be logged in a run group in wandb, and from there the visualizations will be made

In [ ]:
%nbdev_export experiments
@call_parse
def x_sequence_length(min_rp:Param("Minimum resampling period", int)=1,
                      max_rp:Param("Maximum resampling period", int)=10,
                      rep_times:Param("Number of times ru run per rp", int)=10,
                      nbs_folder:Param("Notebook folder path", str)='.'):
    for x in range(min_rp, max_rp+1):
        for _ in range(rep_times):
            pm.execute_notebook(input_path=f'{Path(nbs_folder)}/_motion_classification_time_series.ipynb',
                                output_path='--',
                                parameters=dict(
                                    use_wandb=True,
                                    wandb_group='x_sequence_length',
                                    resampling_period=x
                                ))

In [ ]:
%%capture
x_sequence_length(max_rp=2, rep_times=1)

### Analyse how the inclusion of more and more data helps to predict better the motions of the next epsilon

In [ ]:
%nbdev_export experiments
@call_parse
def x_next_epsilon(nbs_folder:Param("Notebook folder path", str)='.'):
    base_path = Path.home()
    ds_path = base_path/'data/fli_rotnum/model_1'
    fnames = [
        (ds_path/'eps=0.021/Poincare.plt', ds_path/'eps=0.021/index_fli_freqana.plt'),
        (ds_path/'eps=0.022/Poincare.plt', ds_path/'eps=0.022/index_fli_freqana.plt'), 
        (ds_path/'eps=0.023/Poincare.plt', ds_path/'eps=0.023/index_fli_freqana.plt'), 
        (ds_path/'eps=0.024/Poincare.plt', ds_path/'eps=0.024/index_fli_freqana.plt'), 
        (ds_path/'eps=0.025/Poincare.plt', ds_path/'eps=0.025/index_fli_freqana.plt'), 
    ]
    for i in range(2, len(fnames)+1):
        x_input = fnames[:i]
        val_dataset = x_input[-1][0].parent.name
        pm.execute_notebook(input_path=f'{Path(nbs_folder)}/_motion_classification_time_series.ipynb',
                            output_path='--',
                            parameters=dict(
                                use_wandb=True,
                                wandb_group='x_next_epsilon',
                                #fnames=x_input,
                                val_dataset = val_dataset
                            ))

## Analyse the effect of the labelling method

In [ ]:
%nbdev_export experiments
@call_parse
def x_labelling_method(rep_times:Param("Number of times to run per experiment", int)=10,
                      nbs_folder:Param("Notebook folder path", str)='.'):
    for lbl_method in ['fli', 'freq_ana', 'birkhoff_avg']:
        for _ in range(rep_times):
            pm.execute_notebook(input_path=f'{Path(nbs_folder)}/_motion_classification_time_series.ipynb',
                                output_path='--',
                                parameters=dict(
                                    use_wandb=True,
                                    wandb_group='x_labelling_method',
                                    labelling_method=lbl_method
                                ))